# Climate models, the global energy budget, and Fun with Python

This notebook is part of [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook) by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.

____________
<a id='section1'></a>

## 1. What is a Climate Model?
____________

First, some thoughts on modeling from [xkcd](https://xkcd.com)

![physicists](https://imgs.xkcd.com/comics/physicists.png)

Let's be a little pedantic and decompose that question:

- what is Climate?
- what is a Model?

**Climate** is

- statistics of weather, e.g. space and time averages of temperature and precip.
- (statistics might also mean higher-order stats: variability etc)

A **model** is

 - not easy to define!

Wikipedia: http://en.wikipedia.org/wiki/Conceptual_model

> In the most general sense, a model is anything used in any way to represent anything else. Some models are physical objects, for instance, a toy model which may be assembled, and may even be made to work like the object it represents. Whereas, a conceptual model is a model made of the composition of concepts, that thus exists only in the mind. Conceptual models are used to help us know, understand, or simulate the subject matter they represent.

George E. P. Box (statistician):
> Essentially, all models are wrong, but some are useful.”

From the Climate Modelling Primer, 4th ed (McGuffie and Henderson-Sellers):

> In the broadest sense, models are for learning about the world (in our case, the climate) and the learning takes place in the contruction and the manipulation of the model, as anyone who has watched a child build idealised houses or spaceships with Lego, or built with it themselves, will know.  Climate models are, likewise, idealised representations of a complicated and complex reality through which our understanding of the climate has significantly expanded. All models involve some ignoring, distoring and approximating, but gradually they allow us to build understanding of the system being modelled. A child's Lego construction typically contains the essential elements of the real objects, improves with attention to detail, helps them understand the real world, but is never confused with the real thing.

### A minimal definition of a climate model

*A representation of the exchange of energy between the Earth system and space, and its effects on average surface temperature.*

(what average?) 

Note the focus on **planetary energy budget**. That’s the key to all climate modeling.

____________
<a id='section2'></a>

## 2. The observed global energy budget
____________

The figure below shows current best estimates of the *global, annual mean* energy fluxes through the climate system.

We will look at many of these processes in detail throughout the course.

![Observed global energy flows from Trenberth and Fasullo (2012)](../images/GlobalEnergyBudget.png)

## Things to note:

### On the shortwave side

- global mean albedo is 101.9 W m$^{-2}$ / 341.3 W m$^{-2}$ = 0.299
- Reflection off clouds = 79 W m$^{-2}$
- Off surface = 23 W m$^{-2}$
    - 3 times as much reflection off clouds as off surface
    
Why??  Think about both areas of ice and snow, and the fact that sunlight has to travel through cloudy atmosphere to get to the ice and snow. Also there is some absorption of shortwave by the atmosphere.

- Atmospheric absorption = 78 W m$^{-2}$
(so about the same as reflected by clouds)

QUESTION: Which gases contribute to shortwave absorption?

- O$_3$ and H$_2$O mostly.
- We will look at this later.

### On the longwave side

- Observed emission from the SURFACE is 396 W m$^{-2}$
- very close to the blackbody emission $\sigma T^4$ at $T = 288$ K (the global mean surface temperature).
- BUT emission to space is much smaller = 239 W m$^{-2}$

QUESTION: What do we call this?  (greenhouse effect)

### Look at net numbers…

- Net absorbed = 0.9 W m$^{-2}$
- Why?
- Where is that heat going?

Note, the exchanges of energy between the surface and the atmosphere are complicated, involve a number of different processes. We will look at these more carefully later.

### Additional points:

- Notice that this is a budget of energy, not temperature.
- We will need to discuss the connection between the two
- **Clouds** affect both longwave and shortwave sides of the budget.
- **WATER** is involved in many of the terms: 

    - evaporation
    - latent heating (equal and opposite in the global mean)
    - clouds
    - greenhouse effect
    - atmospheric SW absorption
    - surface reflectivity (ice and snow)

### Discussion point

How might we expect some of the terms in the global energy budget to vary under anthropogenic climate change?

____________

## 3. Using Python to compute emission to space
____________

Suppose the Earth behaves like a **blackbody radiator** with effective global mean **emission temperature $T_e$**.

Then

$$ OLR = \epsilon \sigma T_e^4 $$

where:-

- OLR = "Outgoing Longwave Radiation", shown in the above diagram to be for the earth, 238.5 W m$^{-2}$

- $\epsilon$ is 1 which denotes a perfect black body

- $\sigma = 5.67 \times 10{-8}$ W m$^{-2}$ K$^{-4}$ the Stefan-Boltzmann constant

### Re-arranging we can calculate the emission temperature $T_e$ as:-

$$ T_e = \left(\frac{\text{OLR}}{\sigma} \right)^{\frac{1}{4}} $$

First just use Python like a hand calculator to calculate $T_e$ iteractively:

In [2]:
sigma = (5.67 * 10**-8)
Te = (238.5 / sigma) ** 0.25
Te

254.66914551158553

### Theoretical Black Box Earth Temperature is Less than Actual

The actual global mean temperature is said to be 14.9 centigrade which is 288 $K$ higher than the 255 $K$ of the black body model. This is because not all of the emitted radiation makes it out to space due to the greenhouse effect.

### A simple greenhouse model

We will model the OLR as

$$ \text{OLR} = \tau \sigma T_s^4 $$

where $\tau$ is a number we will call the **transmissivity** of the atmosphere.


Rearranging:

$$ \tau = \frac{\text{OLR}}{\sigma T_s^4} $$

In [3]:
tau = 238.5 / (sigma * 288**4)
tau

0.6114139923687018

Try calculating OLR for a warmer Earth at 292 K:

In [15]:
OLR = tau * (sigma * 292**4)
OLR

252.02860648282103

Naturally the emission to space is higher. By how much has it increased for this 4 degree warming?

In [16]:
OLR - 238.5

13.528606482821033

Answer: 13.5 W m$^{-2}$. Okay but this is tedious and prone to error.
What we really want to do is **define a reusable function**

In [26]:
def OLR(temp):
    sigma = (5.67 * 10**-8)
    tau = 238.5 / sigma / 288**4
    olr = tau * (sigma * temp**4)
    return olr

In [27]:
print(OLR(288), OLR(292), OLR(292)-OLR(288))

238.49999999999997 252.02860648282103 13.528606482821061


### Arrays with `numpy`

Now let’s try some array calculations:

In [22]:
import numpy as np
T = np.linspace(230, 300, 10)
print(T)

[230.         237.77777778 245.55555556 253.33333333 261.11111111
 268.88888889 276.66666667 284.44444444 292.22222222 300.        ]


- We have just created an array object. 
- The `linspace` function creates an array of numbers evenly spaced between the start and end points. 
- The third argument tells Python how many elements we want.

We will use the `numpy` package all the time. It is the basic workhorse of scientific computing with Python. We can't do much with arrays of numbers.

Does our `OLR` function work on an array of temperature values?

In [23]:
OLR(T)

array([ 97.01280716, 110.81607992, 126.04243493, 142.7864768 ,
       161.1458549 , 181.22126332, 203.11644092, 226.9381713 ,
       252.7962828 , 280.80364851])

____________

## 4. Summary
____________

- **Climate** is essentially **statistics of weather**.
- The planet **warms up** or **cools down** in response to **differences** between **energy absorbed from the sun** and **energy emitted to space**.
- A **climate model** represents (mathematically of numerically) these **exchanges of energy** between the **Earth system** and **space**.
- The observed emission to space or **Outgoing Longwave Radiation** is consistent with an emission temperature $T_e = 255$ K -- **much colder than Earth's surface**.
- This is evidence of the **greenhouse effect**.
- We adopted a very simple greenhouse model, assuming a fixed *transmissivity* $\tau$ for the atmosphere. 
- $\tau$ conceptually represents the *fraction of the emission from the **surface** that makes it all the way to space.* It is a number less than 1.
- Python is fun and useful.

____________

## Credits

This notebook is part of [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook), an open-source textbook developed and maintained by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.

It is licensed for free and open consumption under the
[Creative Commons Attribution 4.0 International (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/) license.

Development of these notes and the [climlab software](https://github.com/brian-rose/climlab) is partially supported by the National Science Foundation under award AGS-1455071 to Brian Rose. Any opinions, findings, conclusions or recommendations expressed here are mine and do not necessarily reflect the views of the National Science Foundation.
____________